In [1]:
import keras
import numpy as np
from neuralStellar import *
import pymc3 as pm

Using TensorFlow backend.


In [2]:
m1=NNmodel('evo',['mass','age','feh','MLT'],['L','Teff','delnu'])
m1.loadModel('Ran_files/small_grid_28/small_grid_model.h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [4]:
rawdata=[]
for line in open('M67_data.txt','r').read().split('\n'):
    rawdata.append(line.split(' '))
print(rawdata[0])
data=[]
for stardata in rawdata[1:]:
    if len(stardata)==len(rawdata[0]):
        data.append(stardata)

def readTable(index, data):
    obss=[]
    errs=[]
    for stardata in data:
        obs=float(stardata[index].split('(')[0])
        err=stardata[index].split('(')[1][:-1]
        if '.' not in err:
            err=float('0.'+err)
        else: err=float(err)
        obss.append(obs)
        errs.append(err)
    return np.array(obss), np.array(errs)
        
ages, age_sigmas =  readTable(-1,data)
delnus, delnu_errs = readTable(4,data)
Teffs, Teff_errs = readTable(5,data)
    
Xobs=ages
sigma_obs=np.average(age_sigmas)
N=len(ages)
print(np.mean(ages),sigma_obs)

['EPIC_ID', 'WOCS_ID', 'Class', 'Î½max', 'âˆ†Î½', 'Teff/K', 'Rsc', 'Msc', 'log_gsc', 'Rcorr', 'Mcorr', 'Rgrid', 'Mgrid', 'log_ggrid', 'Agegrid']
4.787096774193548 1.5612903225806454


In [27]:
m1.getWeights()

weights=m1.model.get_weights()
def manualPredict(inputs):
    no_hidden_layers = len(weights)/2-1
    #input shape = 1D array with length as number of NN inputs
    xx=inputs
    for i in np.arange(0,no_hidden_layers)*2:
        i=int(i)
        xx=T.nnet.relu(pm.math.dot(xx,weights[i])+weights[i+1],0)
    xx=pm.math.dot(xx,weights[-2])+weights[-1]
    return xx.T

#mass mean - lognormal
M_mu_mu = 1
M_mu_sigma = 1
#mass sigma - half normal
M_sigma_sigma = 5

#age mean - lognormal
Age_mu_mu = 4.5
Age_mu_sigma = 1
#age sigma - half normal
Age_sigma_sigma = 5

#feh mean - normal
feh_mu_mu = 0
feh_mu_sigma = 1
#feh sigma - half normal
feh_sigma_sigma = 1

#alpha MLT mean - normal
MLT_mu_mu = 2
MLT_mu_sigma = 1
#alpha MLT sigma - half normal
MLT_sigma_sigma = 0.5

N = len(data)

model = pm.Model()
with model:
    M_mu = pm.Lognormal('mean_mass', np.log10(M_mu_mu), M_mu_sigma)
    M_sigma = pm.HalfNormal('spread_mass', M_sigma_sigma)
    
    Age_mu = pm.Lognormal('mean_age', np.log10(Age_mu_mu), Age_mu_sigma)
    Age_sigma = pm.HalfNormal('spread_age', Age_sigma_sigma)
    
    feh_mu = pm.Normal('mean_feh', feh_mu_mu, feh_mu_sigma)
    feh_sigma = pm.HalfNormal('spread_feh', feh_sigma_sigma)
    
    MLT_mu = pm.Normal('mean_MLT', MLT_mu_mu, MLT_mu_sigma)
    MLT_sigma = pm.HalfNormal('spread_MLT', MLT_sigma_sigma)
    
    M = pm.Normal('mass', M_mu, M_sigma, shape=N)
    Age = pm.Normal('age', Age_mu, Age_sigma, shape=N)
    feh = pm.Normal('feh', feh_mu, feh_sigma, shape=N)
    MLT = pm.Normal('MLT', MLT_mu, MLT_sigma, shape=N)
    
    obs = pm.Deterministic('obs',manualPredict([np.log10(M), np.log10(Age), feh, MLT]))
    
    #obs_L = pm.Normal('obs_L',10**obs[0],0.1, observed=1.0)
    obs_Teff = pm.Normal('obs_Teff',10**obs[1],Teff_errs, observed=Teffs)
    obs_delnu = pm.Normal('obs_delnu',10**obs[2],delnu_errs, observed=delnus)

with model:
    trace = pm.sample()

ValueError: shapes (4,31) and (4,128) not aligned: 31 (dim 1) != 4 (dim 0)

In [22]:
a=np.array([[1,2,3,4],[10,20,30,40]])
b=np.array([[1,1],[2,2],[3,3],[4,4]])
print(np.dot(b.T,a.T))

[[ 30 300]
 [ 30 300]]
